In [1]:
import json
from jsonpath import jsonpath
import pandas as pd
import re
import os
import glob
import pdfkit
import imgkit
import urllib.request
from selenium import webdriver 
import time
import os.path
import multiprocessing as mp
import wechatsogou
from cachelib import SimpleCache

# pd.set_option('display.max_colwidth', -1)

In [2]:
# 提取近 10 篇公众号文章信息
# captcha_break_time为验证码输入错误的重试次数，默认为1
# ws_api = wechatsogou.WechatSogouAPI(captcha_break_time=3)
# 公众号名称
# gzh_name = '数海随记'
# 将该公众号最近10篇文章信息以字典形式返回
# data_10 = ws_api.get_gzh_article_by_history(gzh_name)
# data_10

In [3]:
# 遍历文件夹下的 JSON 文件，循环读取内容
folder_path = './我的公众号JSON'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
file_form = '*.json'
files_name = os.path.join(folder_path,file_form)
all_files = glob.glob(files_name)
all_files

['./我的公众号JSON\\26_.json',
 './我的公众号JSON\\34_.json',
 './我的公众号JSON\\35_.json',
 './我的公众号JSON\\36_.json']

In [4]:
# 获取单个 JSON 文件中的 文章标题、网页链接  
df = pd.DataFrame(columns=['文章标题','网页链接'])

for file in all_files:

    title_list = []
    content_url_list = []

    try:
        json_data = json.load(open(file,'r',encoding='utf-8'))
        data_list = json.loads(json_data['general_msg_list'])['list']
        for data in data_list:           
            title = data['app_msg_ext_info']['title']
            content_url = data['app_msg_ext_info']['content_url']                   
            title_list.append(title)
            content_url_list.append(content_url)
    except:
        pass

    df_json = pd.DataFrame({'文章标题':title_list,'网页链接':content_url_list})
    df = pd.concat([df,df_json],ignore_index=True)

df['文章标题'] = df['文章标题'].str.replace('|','-')
df = df[df.文章标题.str.len() < 40]

folder_name = './我的公众号HTML文件'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
html_list = []
ws_api = wechatsogou.WechatSogouAPI(captcha_break_time=3)
for i,j in zip(df['网页链接'],df['文章标题']):
    content_info = ws_api.get_article_content(i)
    html_code = content_info['content_html']
    
    html = f'''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="referrer" content="same-origin">
        <title>{j}</title>
    </head>
    <!-- <link href="my.css" rel="stylesheet" type="text/css" /> -->
    <link href="my.css" rel="stylesheet" type="text/css" />
    <body>
    <h2 style="text-align: center;font-weight: 400;">{j}</h2>
    {html_code}
    </body>
    </html>
    '''
    html_list.append(html)
    
    format_name = j + '.html'
    file_name = os.path.join(folder_name,format_name)
    with open(file_name,'wb') as f:
        f.write(html.encode(encoding='utf-8'))

df['网页源码'] = html_list
df

D:\software\anaconda\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.


,文章标题,网页链接,网页源码
0,猴子新书《数据分析思维》读后笔记,http://mp.weixin.qq.com/s?__biz=MzU0MTkyODQwMA...,"\n <!DOCTYPE html>\n <html lang=""en"">\n ..."
1,浅谈 Python 中的深浅拷贝,http://mp.weixin.qq.com/s?__biz=MzU0MTkyODQwMA...,"\n <!DOCTYPE html>\n <html lang=""en"">\n ..."
2,小心 Python 中函数传参的“陷阱”,http://mp.weixin.qq.com/s?__biz=MzU0MTkyODQwMA...,"\n <!DOCTYPE html>\n <html lang=""en"">\n ..."
3,有趣的 python 考题，你也来挑战一下吧,http://mp.weixin.qq.com/s?__biz=MzU0MTkyODQwMA...,"\n <!DOCTYPE html>\n <html lang=""en"">\n ..."
4,提升生产力，在 Ubuntu 下配置 oh-my-zsh 打造完美终端,http://mp.weixin.qq.com/s?__biz=MzU0MTkyODQwMA...,"\n <!DOCTYPE html>\n <html lang=""en"">\n ..."
5,风控建模通用步骤,http://mp.weixin.qq.com/s?__biz=MzU0MTkyODQwMA...,"\n <!DOCTYPE html>\n <html lang=""en"">\n ..."
6,做要好运营工作，要点笔记需记牢,http://mp.weixin.qq.com/s?__biz=MzU0MTkyODQwMA...,"\n <!DOCTYPE html>\n <html lang=""en"">\n ..."
7,Python实战数据分析系列之一 - 为什么销售额会减少,http://mp.weixin.qq.com/s?__biz=MzU0MTkyODQwMA...,"\n <!DOCTYPE html>\n <html lang=""en"">\n ..."
8,为你呈上四千字长文笔记，SQL 进阶要点总结,http://mp.weixin.qq.com/s?__biz=MzU0MTkyODQwMA...,"\n <!DOCTYPE html>\n <html lang=""en"">\n ..."
9,听说滴滴的SQL有点难度，看我把题目解决掉,http://mp.weixin.qq.com/s?__biz=MzU0MTkyODQwMA...,"\n <!DOCTYPE html>\n <html lang=""en"">\n ..."


In [5]:
# 循环读取 df 中的网址，对网页进行 PDF 转化
folder_name = './我的公众号PDF'

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
config = pdfkit.configuration(wkhtmltopdf=r'D:\software\wkhtmltopdf\bin\wkhtmltopdf.exe')
wkhtmltopdf_options = {'enable-local-file-access': None,'quiet': ''}

for url,title in zip(df['网页链接'],df['文章标题']):
    format_name = title + '_图片缺失.pdf'
    file_name = os.path.join(folder_name,format_name)
    pdfkit.from_url(url,file_name,configuration=config)
print('生成 PDF 文件成功！！')

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                        
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                       

In [6]:
# 循环读取 df 中的网址，对网页进行图片转化
folder_name = './我的公众号图片'

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
config = imgkit.config(wkhtmltoimage=r'D:\software\wkhtmltopdf\bin\wkhtmltoimage.exe')
# wkhtmltopdf_options = {'enable-local-file-access': None}
options = {"encoding": "UTF-8",'quiet': ''}

for url,title in zip(df['网页链接'],df['文章标题']):
    format_name = title + '_图片缺失.jpg'
    file_name = os.path.join(folder_name,format_name)
    imgkit.from_url(url,file_name,config=config,options=options)
print('生成图片成功！！')

生成图片成功！！


In [7]:
# 循环读取 df 中的网址，保存为本地网页
def getHtml(url):
 html = urllib.request.urlopen(url).read()
 return html
 
def saveHtml(file_name, file_content):
        
    with open(file_name + "_图片缺失.html", "wb") as f:
        f.write(file_content)
        
folder_name = './我的公众号网页'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
for url,title in zip(df['网页链接'],df['文章标题']):
    file_name = os.path.join(folder_name,title)
    html = getHtml(url)
    saveHtml(file_name,html)
    print(f'{title} 网页保存成功！！')
print('所有网页保存完毕！！')

猴子新书《数据分析思维》读后笔记 网页保存成功！！
浅谈 Python 中的深浅拷贝 网页保存成功！！
小心 Python 中函数传参的“陷阱” 网页保存成功！！
有趣的 python 考题，你也来挑战一下吧 网页保存成功！！
提升生产力，在 Ubuntu 下配置 oh-my-zsh 打造完美终端 网页保存成功！！
风控建模通用步骤 网页保存成功！！
做要好运营工作，要点笔记需记牢 网页保存成功！！
Python实战数据分析系列之一 - 为什么销售额会减少 网页保存成功！！
为你呈上四千字长文笔记，SQL 进阶要点总结 网页保存成功！！
听说滴滴的SQL有点难度，看我把题目解决掉 网页保存成功！！
为你呈上五千字长文笔记，SQL 入门要点总结 网页保存成功！！
面对为所欲为的手工输入数据，看我来处理成可以分析的格式 网页保存成功！！
关于排名问题的一切，你想知道的都在这里 网页保存成功！！
分组合并拆分文本问题，看我用4种方法搞定它 网页保存成功！！
效率利器 - 不会作图到底有多可怕，有了这款工具你还怕啥？ 网页保存成功！！
协同编程 —— Git 版本控制操作入门指引 网页保存成功！！
对 Python 使用“影分身之术” — 虚拟环境设置方法的梳理 网页保存成功！！
化整为零的魔法师 — 自动将多张图片合成 PDF 文档 网页保存成功！！
自己动手，丰衣足食 — 让你边喝茶边获得“报告收集者”成就的方法 网页保存成功！！
揭秘“啤酒与尿布”的奥秘 — 利用 python 实现购物小票关联分析 网页保存成功！！
百度、腾讯、讯飞API哪家强 — 对三大主流人工智能平台横向测评 网页保存成功！！
所有网页保存完毕！！


In [8]:
# 使用 selenium 将网页转换为图片
def screen_shot(url,png_name):
    # 使用webdirver.PhantomJS()方法新建一个phantomjs的对象，这里会使用到phantomjs.exe，环境变量path中找不到phantomjs.exe，则会报错
    brower = webdriver.PhantomJS(executable_path=r'D:/software/phantomjs/phantomjs-2.1.1-windows/bin/phantomjs.exe')
    # 使用get()方法，打开指定页面。注意这里是phantomjs是无界面的，所以不会有任何页面显示
    brower.get(url)
    # 设置phantomjs浏览器全屏显示
    time.sleep(5)
    brower.maximize_window()
    # 使用save_screenshot将浏览器正文部分截图，即使正文本分无法一页显示完全，save_screenshot也可以完全截图
    brower.save_screenshot(png_name)
    # 关闭phantomjs浏览器，不要忽略了这一步，否则你会在任务浏览器中发现许多任务
    brower.close()

folder_name = './我的公众号网页截图'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
for url,title in zip(df['网页链接'],df['文章标题']):
    format_name = title + '_图片缺失.png'
    file_name = os.path.join(folder_name,format_name)
    screen_shot(url,file_name)
    print(f'{title} 网页截图保存成功！！')
print('所有网页截图保存完毕！！')

D:\software\anaconda\lib\site-packages\selenium\webdriver\phantomjs\webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


猴子新书《数据分析思维》读后笔记 网页截图保存成功！！
浅谈 Python 中的深浅拷贝 网页截图保存成功！！
小心 Python 中函数传参的“陷阱” 网页截图保存成功！！
有趣的 python 考题，你也来挑战一下吧 网页截图保存成功！！
提升生产力，在 Ubuntu 下配置 oh-my-zsh 打造完美终端 网页截图保存成功！！
风控建模通用步骤 网页截图保存成功！！
做要好运营工作，要点笔记需记牢 网页截图保存成功！！
Python实战数据分析系列之一 - 为什么销售额会减少 网页截图保存成功！！
为你呈上四千字长文笔记，SQL 进阶要点总结 网页截图保存成功！！
听说滴滴的SQL有点难度，看我把题目解决掉 网页截图保存成功！！
为你呈上五千字长文笔记，SQL 入门要点总结 网页截图保存成功！！
面对为所欲为的手工输入数据，看我来处理成可以分析的格式 网页截图保存成功！！
关于排名问题的一切，你想知道的都在这里 网页截图保存成功！！
分组合并拆分文本问题，看我用4种方法搞定它 网页截图保存成功！！
效率利器 - 不会作图到底有多可怕，有了这款工具你还怕啥？ 网页截图保存成功！！
协同编程 —— Git 版本控制操作入门指引 网页截图保存成功！！
对 Python 使用“影分身之术” — 虚拟环境设置方法的梳理 网页截图保存成功！！
化整为零的魔法师 — 自动将多张图片合成 PDF 文档 网页截图保存成功！！
自己动手，丰衣足食 — 让你边喝茶边获得“报告收集者”成就的方法 网页截图保存成功！！
揭秘“啤酒与尿布”的奥秘 — 利用 python 实现购物小票关联分析 网页截图保存成功！！
百度、腾讯、讯飞API哪家强 — 对三大主流人工智能平台横向测评 网页截图保存成功！！
所有网页截图保存完毕！！


In [9]:
# 循环读取 df 中的网址，对网页进行 PDF 转化
folder_name = './我的公众号PDF'

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
config = pdfkit.configuration(wkhtmltopdf=r'D:\software\wkhtmltopdf\bin\wkhtmltopdf.exe')
wkhtmltopdf_options = {'enable-local-file-access': None,'quiet': ''}
css = './my.css'

for html,title in zip(df['网页源码'],df['文章标题']):
    format_name = title + '_完美版.pdf'
    file_name = os.path.join(folder_name,format_name)
    try:
        pdfkit.from_string(html,file_name,configuration=config,css=css)
    except Exception as e:
        print(str(e))
        pass
print('生成 PDF 文件成功！！')

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                         

In [10]:
# 循环读取 df 中的网址，对网页进行图片转化
folder_name = './我的公众号图片'

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
config = imgkit.config(wkhtmltoimage=r'D:\software\wkhtmltopdf\bin\wkhtmltoimage.exe')
# wkhtmltopdf_options = {'enable-local-file-access': None}
options = {"encoding": "UTF-8",'quiet': ''}
css = './my.css'

for url,title in zip(df['网页链接'],df['文章标题']):
    format_name = title + '_完美版.jpg'
    file_name = os.path.join(folder_name,format_name)
    html_file = title + '.html'
    html_name = os.path.join('./我的公众号HTML文件',html_file)
    try:
        imgkit.from_file(html_name,file_name,config=config,options=options,css=css)
    except Exception as e:
        print(str(e))
        pass
print('生成图片成功！！')

wkhtmltoimage exited with non-zero code 1. error:
QPainter::begin: Paint device returned engine == 0, type: 3
QPainter::translate: Painter not active
QPainter::setRenderHint: Painter must be active to set rendering hints
QPainter::setBrush: Painter not active
QPainter::pen: Painter not active
QPainter::setPen: Painter not active
QPainter::save: Painter not active
QPainter::setClipRect: Painter not active
QPainter::translate: Painter not active
QPainter::translate: Painter not active
QPainter::setClipRect: Painter not active
QPainter::setCompositionMode: Painter not active
QPainter::setCompositionMode: Painter not active
QPainter::combinedTransform: Painter not active
QPainter::pen: Painter not active
QPainter::setPen: Painter not active
QPainter::setBrush: Painter not active
QPainter::setRenderHint: Painter must be active to set rendering hints
QPainter::setRenderHint: Painter must be active to set rendering hints
QPainter::drawRects: Painter not active
QPainter::setRenderHint: Painter